In [28]:
import requests
import pandas as pd
import re
#import enchant
import numpy as np
from nltk.corpus import words

In [3]:
# This "interval" "definition" is terrible. It even includes species,that, supposedly are geological time intervals
#r = requests.get("https://macrostrat.org/api/defs/intervals?all").json()
#time_intervals = [x["name"].replace("\t", "") for x in r["success"]["data"]]
r = requests.get("https://macrostrat.org/api/defs/strat_names?all").json()
#r = requests.get("https://macrostrat.org/api/defs/strat_names?rank=Fm").json()


In [4]:
pbdb_stratigraphic_names = requests.get("https://paleobiodb.org/data1.2/occs/strata.json?datainfo&rowcount&interval=Phanerozoic&rank=Formation").json()

In [5]:
strat_names = [x["strat_name_long"] for x in r["success"]["data"]]
print(len(r["success"]["data"]))
#[x["strat_name_long"] for x in r["success"]["data"][15000:15050]]
#pbdb_stratigraphic_names["records"][1000]
#strat_names_pbdb = 

45619


In [10]:
with open("data/species_names_bryozoa.txt", "r") as myfile:
	taxa = myfile.read().split("\n")

taxa2 = pd.read_csv("data/CoL_bryozoa.csv")
taxa2 = [' '.join(x.split('  ')[0:2]) for x in list(taxa2["Scientific name"])]
taxa3 = taxa + taxa2

## Problem: some species are written in a bad format, e.g.
## 
## Chaperiopsis (Clipeochaperia) frontalis
##
# ambiguous = [x for x in taxa3 if "(" in x]



In [17]:
philbock_taxa = pd.read_csv("data/philbock_species.csv", sep = "\t", dtype=str)

taxa = [x.capitalize() for x in philbock_taxa.Name.dropna()]
taxa = [x for x in taxa if "subsequent" not in x]

def is_ascii(s):
    return all(ord(c) < 128 for c in s)

#[x for x in philbocktaxa2 if ")" in x]
#[x for x in philbocktaxa2 if "?" in x]
#[x for x in philbocktaxa2 if not is_ascii(x)]
#[x for x in philbocktaxa2 if len(x.split(' ')) > 2][2000:]
#[x.replace(" [sic]", "") for x in taxa]
#[x for x in taxa if "bouei minute" in x]

#re.search("No subsequent", taxa)
[x for x in taxa if " strata" in x]


['Fenestella strata']

In [18]:
def cleanup(genera):
    """
    Function to remove any of the matches in "flags2" from the string input
    """
    flags2 = {"(?)", "*", " [sic]", "?", " cf.", " n. sp.", " 1", 
              " l.", " s.", " j.", 
              " sp.", " aff.", " ex gr.", " [1938]", 
              "\xa0", "\t", " [sic, read costulatus]", " [diastopora]", '&', 
              ' f.', ' l.', ' c.', ' s.',' ex.', 'a.', 'b.', 'grp.',
              ' genus', ' indet.', ' egenus'}
    rep = {x: "" for x in flags2}
    rep = dict((re.escape(k), v) for k, v in rep.items())

    pattern = re.compile("|".join(rep.keys()))
    res = map(lambda genus: pattern.sub(lambda m: rep[re.escape(m.group(0))], genus), (x for x in genera if len(x) > 1))
    return(res)

philbocktaxa = []

taxa = cleanup(taxa)

for taxon in taxa:
    if "sp." in taxon:
        taxon = taxon[:pat.search(taxon).end()]
    
    split_taxon = taxon.split()   
    
    pindices = [i for i, p in enumerate(taxon) if p in "()"]

    if len(pindices) > 1:
        if " " in taxon[pindices[0]:pindices[1]+1]:
            taxon = taxon[:pindices[0]] + taxon[pindices[1]+2:]
            
            taxon = " ".join(taxon.split()) ## Normalize spacing
            philbocktaxa.append(taxon)
            
        else:
            first = taxon.replace(taxon[pindices[0]:pindices[1]+2], "")
            second = taxon[pindices[0]+1:].replace(")", "")
        
            philbocktaxa.append(first.capitalize())
            philbocktaxa.append(second.capitalize())  
        
    
    else:
        taxon = " ".join(taxon.split()) ## Normalize spacing
        philbocktaxa.append(taxon)
        
def nonascii_fix(x):
    x = x.replace("Macropora ﬁlifera", "Macropora filifera")
    x = x.replace("Ralﬁna originalis", "Ralfina originalis")
    x = x.replace("‘hastingsia’ whitteni", "hastingsia whitteni")
    return(x.replace("Biflustra’ lamellosa", "Biflustra lamellosa"))

philbocktaxa2 = [nonascii_fix(x).capitalize() for x in philbocktaxa]
        

philbocktaxa3 = []

for x in philbocktaxa2:
    philbocktaxa3.append(x)
    
    k = x.split()
    if len(k) > 2:
        philbocktaxa3.append(' '.join(k[0:2]))
        
        

taxa4 = set(x for x in set(taxa3 + philbocktaxa3) if x not in ["Sic", "Delete", "Ignore", "More work", "Zzcomment"])

#sorted([x.split()[-1] for x in philbocktaxa3], key = len)[0:30]

#print(philbocktaxa3[0:10])


k = sorted([x.split()[-1] for x in taxa4 if " " in x], key = len)[0:1000]

In [30]:

## Dictionary for english (US) words
#d = enchant.Dict("en_US")

#nltk.download("words")
english_words = set(words.words())

In [38]:
genera = {x.split(" ")[0] for x in taxa4}
print("Number of genera:", len(genera))

english_genera = {x for x in genera if x in english_words}

print(english_genera)


Number of genera: 3121
{'Plumatella', 'Orbitolites', 'Paulina', 'Gargantua', 'Galeopsis', 'Lunularia', 'Archimedes', 'Boreas', 'Tubularia', 'Avicularia', 'Lunulites', 'Dictyonema', 'Porella', 'Corallina', 'Favosites', 'Campanularia', 'Heliolites', 'Alcyonium', 'Adeona', 'Stromatopora', 'Tubipora', 'Arachnoidea', 'Sertularia', 'Hydra', 'Oculina', 'Vesicularia', 'Crepis', 'Tubulipora', 'Chaetites', 'Millepora', 'Monticulipora', 'Membranipora', 'Paludicella', 'Stenophragma', 'Lichenopora', 'Cellepora', 'Lepralia', 'Diplotaxis', 'Alveolites', 'Electra', 'Madrepora', 'Fedora', 'Retepora', 'Cristatella', 'Mastigophora', 'Isis', 'Fungia', 'Fistularia', 'Emma', 'Flustra', 'Chaetetes', 'Ulva', 'Oncidium', 'Acropora', 'Hydractinia', 'Salicornia', 'Gorgonia'}


In [39]:
# Pick some of the above genera, remove them
bad_genera = {"Emma", "Hydra", "Isis", "Ulva", "Avicularia", "Boreas"}

genera_minus_english = genera.difference(bad_genera)
print("Number of genera, minus english words:", len(genera_minus_english))

with open("data/genera.csv", "w") as f:
    f.write("\n".join(genera_minus_english))

Number of genera, minus english words: 3115


In [40]:
## If the epithet is in the english dictionary, add to set
english_epithets = {x.split()[-1] for x in taxa4 if x.split()[-1] in english_words}

#{1,2,5}.union({"a"}) ## join two sets

#{x for x in intervals if x in english_words}


## Geochronological time intervals

First from http://www.stratigraphy.org/bak/geowhen/geolist.html
retrieved 26 march 2018

45619

In [44]:
bad = {"Early", "Late", "Lower", "Middle", "Upper", "III", "II", "VI", "V", "I", "(475", "Stages", "Displayed"}

df = pd.read_csv("data/geolist.txt", sep = "\t", header = None)
k = [x.split(" ")[3:5] for x in df[0]]
stages = [[m for m in x if m not in bad] for x in k]
stages = [[m for m in x if m] for x in stages if x]
stages = {' '.join(x) for x in stages}

#{x for x in stages if d.check(x)}


In [70]:
with open("data/names_geological.txt", "r") as myfile:
    intervals = myfile.read().split("\n")
if intervals[-1] == "":
    intervals.pop()

intervals = set(intervals).union(stages)
rules = []

# Strings (or tokens) of species epithets that are not to be added as regexpattern 
flags_pink = {"avicularia", "ta", "moe", "ab", "to", "mui", "lui", 
              'sa', "de", "a", "b", "gay", "genus", "comes",
              "inferior", "biota", "bella", "sinus", "No", "rostrum", "work",
              "major", "minor", "minimum", "fertile", "More", "No", "ordo", "index", "More", "more",
              "Was", "st.", "strata", "foraminifera", "Was", "nuda", "canui", 
              "richteri", "delta", "gigas", "foramina", "fossula", "minima", "americana", "voigti"}
flags = flags_pink.union(english_epithets)


override = "ORGANIZATION,PERSON,LOCATION,MISC,NUMBER,MONEY,PERCENT,DATE,TIME"
for species in taxa4:
    k = species.split(" ")

    
    if len(k) > 1:
        if k[1] not in flags_pink and k[-1] not in flags:
            if k[1] == "to":
                print("ASDHASD")
            
            rules.append(''.join((k[1], "\t", "TAXA", "\t", override, "\t", "1.0")))
            
            rules.append(''.join((k[0], "\t", "TAXA", "\t", override, "\t", "1.0")))
            rules.append(''.join((k[0], " aff.","\t", "TAXA", "\t", override, "\t", "2.0")))
            rules.append(''.join((k[0], " sp.","\t", "TAXA", "\t", override, "\t", "2.0")))#rules.append("([^\s]+) sp\tTAXA") 

for species in taxa4:
    k = species.split()
    
    if len(k) > 1:
        abbrev_rule = ["[A-Z][.] ",k[1],"\t", "TAXA", "\t", override, "\t", "3.0"]
        rules.append(''.join(abbrev_rule))
        
        abbrev_rule.insert(1, "cf. ")
        rules.append(''.join(abbrev_rule))
        
        abbrev_rule[1] = "aff. "
        rules.append(''.join(abbrev_rule))


for species in taxa4:
    k = species.split()

    if len(k) > 1:
        k.insert(1, "cf.")
        rules.append(''.join((' '.join(k), "\t", "TAXA", "\t", override, "\t", "3.0")))

        k[1] = "aff."
        rules.append(''.join((' '.join(k), "\t", "TAXA", "\t", override, "\t", "3.0")))

        rules.append(''.join((species,"\t", "TAXA", "\t", override, "\t", "3.0")))

    ## Following regex could replace all of the above, except mentions of lone genera (no epithets)
    ## However, StanfordNLP RegexNER does not like it, gives error when annotating named entities.
    ##
    ## rules.append(''.join(["([A-Z][.] |", k[0]," )?(aff |cf |sp |sp nov )?", k[1], "\t", "TAXA", "\t", "2.0"]))

bad_intervals = {"Gulf", "Black", "Early", "Late", "Older", "Oldest", "Arnold", "Canadian", "Cryptic", "Glacial", "Mayan", "Namibian", "Younger",
                        "Primary", "Primordial", "Mayan", "Gallic", "Namibian", "Romanian", "Secondary", "Sicilian", "Warsaw"}
for interval in {x for x in intervals if x not in bad_intervals}:
    if interval == "Arnold":
        print("asd")
        
    for prefix in {"Late", "late", "Early", "early", "Middle", "middle", "upper", "Upper", "Lower", "lower"}:
        rules.append(''.join((prefix, " ", interval,"\t", "INTERVALNAME", "\t", override, "\t", "2.0")))
        rules.append(''.join((interval,"\t", "INTERVALNAME", "\t", override, "\t", "1.0")))
    
for strat_name in strat_names:
    rules.append(''.join((strat_name.replace("+ ", ""),"\t", "STRAT", "\t", override, "\t", "1.0")))

rules.append(''.join(("Recent", "\t", "INTERVALNAME", "\t", override, "\t", "1.0")))
rules.append(''.join(("The Recent", "\t", "INTERVALNAME", "\t", override, "\t", "2.0")))



In [71]:
[x for x in rules if "Arnold" in x]

['Arnold Member\tSTRAT\tORGANIZATION,PERSON,LOCATION,MISC,NUMBER,MONEY,PERCENT,DATE,TIME\t1.0',
 'Arnold River Formation\tSTRAT\tORGANIZATION,PERSON,LOCATION,MISC,NUMBER,MONEY,PERCENT,DATE,TIME\t1.0',
 'Arnold River Formation\tSTRAT\tORGANIZATION,PERSON,LOCATION,MISC,NUMBER,MONEY,PERCENT,DATE,TIME\t1.0',
 'Arnold Sandstone Member\tSTRAT\tORGANIZATION,PERSON,LOCATION,MISC,NUMBER,MONEY,PERCENT,DATE,TIME\t1.0']

# Add formation rules

This line is a TokenRegex, see

https://nlp.stanford.edu/software/tokensregex.html

```
@techreport{tokensregex2014,
  author    = {Chang, Angel X. and Manning, Christopher D.},
  title  = {{TokensRegex}: Defining cascaded regular expressions over tokens},
  number = {CSTR 2014-02},
  institution = {Department of Computer Science, Stanford University},
  year = {2014}
}
```

In [72]:
stratigraphy = "( [{ word:/[A-Z][a-z]+/ }]+? /Dolomite|dolomite|Dolostone|dolostone|Limestone|limestone|Sandstone|sandstone|Shale|shale|Conglomerate|chert|Chert|Group|Formation|formation|Member|Supergroup|supergroup|Bed|Subgroup|subgroup|Fm|Mbr|Gp|SGp/ )"

strat_rule = "\t".join([stratigraphy, "STRAT", override, "3.0"])
rules.append(strat_rule)


# Save it

In [67]:
## Save RegexNER rules to file
with open('output/regexner_rules.txt', mode='w', encoding='utf-8') as myfile:
    myfile.write('\n'.join(line for line in rules))

## Some summary

By inspecting the file "ner-training/regexner_rules.txt" we can inspect all of the regex expressions, including their target labels and label priorities (1.0 or 2.0). Here, "TAXA" is used for taxa, and "INTERVALNAME" is used for geological time intervals. We can inspect the first entries:

In [66]:
!head output/regexner_rules.txt
!echo ""
!wc output/regexner_rules.txt

poissonii	TAXA	ORGANIZATION,PERSON,LOCATION,MISC,NUMBER,MONEY,PERCENT,DATE,TIME	1.0
Crepidacantha	TAXA	ORGANIZATION,PERSON,LOCATION,MISC,NUMBER,MONEY,PERCENT,DATE,TIME	1.0
Crepidacantha aff.	TAXA	ORGANIZATION,PERSON,LOCATION,MISC,NUMBER,MONEY,PERCENT,DATE,TIME	2.0
Crepidacantha sp.	TAXA	ORGANIZATION,PERSON,LOCATION,MISC,NUMBER,MONEY,PERCENT,DATE,TIME	2.0
nereidis	TAXA	ORGANIZATION,PERSON,LOCATION,MISC,NUMBER,MONEY,PERCENT,DATE,TIME	1.0
Pinnatopora	TAXA	ORGANIZATION,PERSON,LOCATION,MISC,NUMBER,MONEY,PERCENT,DATE,TIME	1.0
Pinnatopora aff.	TAXA	ORGANIZATION,PERSON,LOCATION,MISC,NUMBER,MONEY,PERCENT,DATE,TIME	2.0
Pinnatopora sp.	TAXA	ORGANIZATION,PERSON,LOCATION,MISC,NUMBER,MONEY,PERCENT,DATE,TIME	2.0
pertusa	TAXA	ORGANIZATION,PERSON,LOCATION,MISC,NUMBER,MONEY,PERCENT,DATE,TIME	1.0
Strebloplax	TAXA	ORGANIZATION,PERSON,LOCATION,MISC,NUMBER,MONEY,PERCENT,DATE,TIME	1.0
""
  496925  2609239 47310468 output/regexner_rules.txt
